In [ ]:
#Imports

In [7]:

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sqlite3 as lite
import sys

low_memory=False

plt.style.use('fivethirtyeight')
sns.set()
%matplotlib inline
#import dataset
data = pd.read_csv('CandidateSummaryActionSenate.csv')
#trimming unecessary columns
ndata = data[['can_nam','can_off', 'can_par_aff','ind_ite_con','ind_uni_con', 'ind_con'  ]]
ndata.columns=['candidate name' ,'running for', 'party', 'individual itemized', 'individual unitemized', "total individual contributions" ]
#prepare for cleaning

ndata[1:10]



,candidate name,running for,party,individual itemized,individual unitemized,total individual contributions
1,"EHLER, CHARLES JOSEPH",S,REP,NaN,NaN,NaN
2,"MOHARIR, MUKUND MARTAND",S,IND,NaN,NaN,NaN
3,"FRAZIER, RYAN LAVAR",S,REP,"$237,080.00","$19,321.00","$256,401.00"
4,"GRASSLEY, CHARLES E SENATOR",S,REP,"$4,414,642.00","$1,218,408.00","$5,633,053.00"
5,"BLUNT, ROY",S,REP,"$6,444,218.00","$414,311.00","$6,658,523.00"
6,"PEARSON, CECIL WELDON JR",S,DEM,NaN,NaN,NaN
7,"GATES, MICHELLE",S,UNK,NaN,NaN,NaN
8,"BEITIKS, MIKELIS VALAINIS",S,IND,"$2,995.00","$2,639.00","$5,634.00"
9,"BOOZMAN, JOHN",S,REP,"$2,040,027.00","$112,514.00","$2,152,542.00"


In [9]:
#Bulk Data processing

In [107]:
# FEC dictates donations above $200 to be itemized, 
# but itemized data can also include values smaller than $200.
# To get the real small donation amount, add the amount of recorded itemized amount under $200 with unitemized amount. 

plt.style.use('fivethirtyeight')
sns.set()
%matplotlib inline
ddata = pd.read_csv('indivs16a.csv', )
ddata[390:397]



,Candidate,donor,amount
390,SMITH,WENDELLMR,51
391,SPEARS,JOHNMR,5/18/2015
392,STAMPER,JOHNHMR,500
393,SamuelBell,C00042366,NaN
394,WALLISON,FRIEDA,250
395,FRAZIER,RYANLAVAR,195
396,WYNNE,ALECMR,50


In [110]:
#cleanup of bad data
mask = ddata.amount.map(lambda x: isinstance(x, str))
frame = ddata[mask]
mask2 = frame.amount.map(lambda x: x.isnumeric())
ddata =frame[mask2]

plt.style.use('fivethirtyeight')
sns.set()
%matplotlib inline
ddata[390:397]




,Candidate,donor,amount
390,SMITH,WENDELLMR,51
392,STAMPER,JOHNHMR,500
394,WALLISON,FRIEDA,250
395,FRAZIER,RYANLAVAR,195
396,WYNNE,ALECMR,50
397,CLARK,THOMASL,300
398,BONSER,KATHLEEN,1000


In [97]:
#getting data smaller than 200
amount= mask3 = ddata.amount.map(lambda x: int(x))
mask4 = amount.map(lambda x: x<=200)
smalldonations = ddata[mask4]
smalldonations[0:10]

,Candidate,donor,amount
0,BERLE,CHRISTIAN,50
1,DELANEY,MARTINMR,25
4,HUMMEL,ROBERTMR,100
5,HUMMEL,ROBERTMR,100
7,HUMMEL,ROBERTMR,100
16,SHERMAN,JONATHANDR,100
18,SIMMS,RICHARDMR,51
19,SKAGGS,JOHNRMR,50
20,SLOTA,ROMUALDDR,51
23,HOLDER,CLAUDIADMS,68
